In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

In [ ]:
!rm -rf /content/outputs /content/results /content/sample_data

In [ ]:
!pip uninstall DeepSentiment > /dev/null

In [ ]:
!mkdir -p /data/glove/  #Create folder to skip download
!pip install --upgrade tqdm > /dev/null
!pip install --upgrade git+https://github.com/NiklasHoltmeyer/sentiment-analysis.git > /dev/null

In [ ]:
!pip install --upgrade tqdm > /dev/null

In [ ]:
from DeepSentiment.Consts import Paths
from DeepSentiment.Networks.Transformer.Model import Model as STModel
from DeepSentiment.Preprocessing.CleanText import CleanText

from sklearn.metrics import confusion_matrix
import pandas as pd 

def getMetrics(result):
    tn, fp, fn, tp = result['tn'], result['fp'], result['fn'], result['tp']
    
    total = tn + fp + fn + tp

    Accuracy = (tp+tn) /total
    Precision = tp/(tp+fp)
    Recall = tp/(tp+fn)
    F1 = 2*Recall*Precision/(Recall+Precision)
    
    return {        
        "Accuracy" : Accuracy,
        "Precision" : Precision,
        "Recall" : Recall,
        "F1" : F1,
        "tn" : tn, 
        "fp" : fp, 
        "fn" : fn, 
        "tp" : tp
    }


def evaluateModel(name, folder, args):
    print("Args")
    print(args)
    splitted = modelName.split("_") #transformer_{model_name}_{entries}_{lr}
    
    model_name, entries, lr = splitted[1], splitted[2], splitted[3]
    model_type = model_name
    stModel = STModel(model_type, model_name)    
    lModel = stModel.load(modelName="model.zip", folder=folderPath)
    stModel.loadData(cleanFN = CleanText().cleanText, args=args)
    lModel.args.lazy_loading = args["lazy_loading"]

    result, _, _ = stModel.validate(stModel.testData, args=args) #result, model_outputs, wrong_predictions
    return getMetrics(result)
    
def metricCSVRow(metric):
    return ";".join([str(v) for v in metric.values()])

def metricHeaderRow(metric):
    return ";".join([k for k in metric.keys()])

In [ ]:
import os
resultPath = "/gdrive/MyDrive/test_1" #todo "/gdrive/MyDrive/test_3"
transformerModelNames = [(x, os.path.join(resultPath, x)) for x in os.listdir(resultPath) if "transformer" in x]

In [ ]:
#trainArgs
validResults = []
errors = []
__results__= []
trainArgs = {"lazy_loading": False, "number_of_training_data_entries" : None}
for modelName, folderPath in transformerModelNames:
  try: 
    results = evaluateModel(name=modelName, folder=folderPath, args=trainArgs)    
    csv = ";".join([modelName, metricCSVRow(results)])
    validResults.append(csv)
    __results__.append(results)
  except Exception as e:
    errors.append((modelName, e))

In [ ]:
errors

In [ ]:
!mkdir -p /content/results/
! $errors > /content/results/errors.txt
errors

In [ ]:
validResults

In [ ]:
! $validResults > /content/results/val_res.txt
validResults

In [ ]:
import io  
from datetime import datetime

metricResult = lambda validResult : ";".join([str(v) for (k, v) in validResult.items()])
metricResultHeader = metricHeaderRow({"Accuracy" : "-", "Precision" : "-", "Recall" : "-", "F1" : "-", "tn" : "-",  "fp" : "-",  "fn" : "-",  "tp" : "-"})

#csvResults = [";".join([validResult[0],validResult[1],";".join([])]) for validResult in validResults]
header = ";".join(["modelName", metricResultHeader])
_results = "\n".join(validResults)

combined = header + '\n' + '\n'+ '\n' + _results
resultDF = pd.read_csv(  io.StringIO(combined)  , sep=";")

resultPath = Paths.RESULTS_BASE + "//result_" + datetime.now().strftime('%Y-%m-%d_%H-%M-%S')  + ".csv"
resultDF.to_csv(resultPath,encoding="utf-8")

In [ ]:
!cp -r /content/results/ /gdrive/MyDrive/result_1_2_all/

In [ ]:
combined

In [ ]:
transformerModelNames